In [1]:
import pyrosetta; pyrosetta.init()
from pyrosetta import *
init()
from pyrosetta.toolbox import pose_from_rcsb
from pyrosetta.toolbox import cleanATOM
from pyrosetta.teaching import *
from pyrosetta import PyMOLMover
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from split_energy import energy_intervals
from split_energy import split_energy
from split_energy import plot
import IPython
from stride import stride
from stride import no_secondary

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python312.mac 2024.35+release.45abd6ab3e15d27b06950672a1bb868f76c03266 2024-08-28T11:40:28] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python312.mac r385 2024.35+release.45abd6ab3e 45abd6ab3e15d27b

In [17]:
polyA = pose_from_sequence('A'*10)

In [18]:
pmm = PyMOLMover()
pmm.keep_history(False)
pmm.apply(polyA)

In [19]:
import math
import random

def randTrial(your_pose):
### BEGIN SOLUTION
    randNum = random.randint(2, your_pose.total_residue())
    currPhi = your_pose.phi(randNum)
    currPsi = your_pose.psi(randNum)
    newPhi = random.gauss(currPhi, 25)
    newPsi = random.gauss(currPsi, 25)
    your_pose.set_phi(randNum,newPhi) 
    your_pose.set_psi(randNum,newPsi)
    pmm.apply(your_pose)
### END SOLUTION
    return your_pose

sfxn = get_fa_scorefxn()

def score(your_pose):
    ### BEGIN SOLUTION
    return sfxn(your_pose)
    ### END SOLUTION

def decision(before_pose, after_pose):
    ### BEGIN SOLUTION
    E = score(after_pose) - score(before_pose)
    if E < 0:
        return after_pose
    elif random.uniform(0, 1) >= math.exp(-E/1):
        return before_pose
    else:
        return after_pose
    ### END SOLUTION

def basic_folding(your_pose):
    """Your basic folding algorithm that completes 100 Monte-Carlo iterations on a given pose"""
    
    lowest_pose = Pose() # Create an empty pose for tracking the lowest energy pose.
    
    ### BEGIN SOLUTION
    for i in range(1000):
        if i == 0:
            lowest_pose.assign(your_pose)
            
        before_pose = Pose()
        before_pose.assign(your_pose) # keep track of pose before random move

        after_pose = Pose()
        after_pose.assign(randTrial(your_pose)) # do random move and store the pose
        
        your_pose.assign(decision(before_pose, after_pose)) # keep the new pose or old pose
        
        if score(your_pose) < score(lowest_pose): # updating lowest pose
            lowest_pose.assign(your_pose)
        
    ### END SOLUTION
    return lowest_pose



core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


In [21]:
basic_folding(polyA)